In [14]:
import time
import numpy
from scipy import spatial as spatial
import pandas as pd
import random


In [21]:
# Create some dummy data
random.seed(1)
y_array = numpy.random.random(10000).reshape(100,100)
x_array = numpy.random.random(10000).reshape(100,100)
points = numpy.random.random(10000).reshape(2,5000)
points

array([[0.45056113, 0.74656658, 0.16520855, ..., 0.88747888, 0.1282837 ,
        0.24395953],
       [0.51347988, 0.73279741, 0.9151298 , ..., 0.49115821, 0.47070187,
        0.64319606]])

In [16]:
# Shoe-horn existing data for entry into KDTree routines
combined_x_y_arrays = numpy.dstack([y_array.ravel(),x_array.ravel()])[0]
points_list = list(points.transpose())


def do_kdtree(combined_x_y_arrays,points):
    mytree = spatial.cKDTree(combined_x_y_arrays)
    dist, indexes = mytree.query(points)
    return dist, indexes

start = time.time()
kd_result = do_kdtree(combined_x_y_arrays,points_list)
end = time.time()
print('Completed in: ',end-start)

Completed in:  0.015316009521484375


In [23]:
combined_x_y_arrays

array([[0.75006454, 0.86847002],
       [0.08569428, 0.64682237],
       [0.89827877, 0.30718365],
       ...,
       [0.92535876, 0.22130957],
       [0.56345967, 0.89419158],
       [0.29852339, 0.36129556]])

In [22]:
points_list

[array([0.51182507, 0.31989802]),
 array([0.48637576, 0.29277517]),
 array([0.60172927, 0.16511992]),
 array([0.20638305, 0.43445144]),
 array([0.2877266 , 0.72925201]),
 array([0.11975938, 0.22724586]),
 array([0.55738484, 0.89611657]),
 array([0.17199436, 0.01273661]),
 array([0.53777293, 0.97613617]),
 array([0.95022631, 0.14848065]),
 array([0.85654445, 0.72530057]),
 array([0.8906072 , 0.01340437]),
 array([0.75388917, 0.60027052]),
 array([0.13581254, 0.27876051]),
 array([0.52295151, 0.76281683]),
 array([0.84143504, 0.29548188]),
 array([0.27600858, 0.80425782]),
 array([0.63075911, 0.68464659]),
 array([0.01128962, 0.51033127]),
 array([0.42695006, 0.78981834]),
 array([0.32883321, 0.2941308 ]),
 array([0.02010536, 0.7105209 ]),
 array([0.90629451, 0.02553872]),
 array([0.73871736, 0.94272951]),
 array([0.51359499, 0.85701674]),
 array([0.91449717, 0.3629804 ]),
 array([0.46409473, 0.46449441]),
 array([0.97807068, 0.87861861]),
 array([0.08344289, 0.87175428]),
 array([0.4240

In [17]:
#
rows = ["dist", "indexes"]
df = pd.DataFrame(data=kd_result, index=rows)

#
nearest_points=combined_x_y_arrays[kd_result[1]]
nearest_points=nearest_points.transpose()

#
df=df.append(pd.DataFrame(nearest_points , index=["nearest_point x","nearest_point y"]))
point_xy=pd.DataFrame(data=points, index=["x","y"])

result = df.append(point_xy).transpose()
result

,dist,indexes,nearest_point x,nearest_point y,x,y
0,0.003754,868.0,0.511637,0.323647,0.511825,0.319898
1,0.004780,576.0,0.481598,0.292902,0.486376,0.292775
2,0.005219,1874.0,0.606934,0.165516,0.601729,0.165120
3,0.002485,6206.0,0.205719,0.432057,0.206383,0.434451
4,0.002768,635.0,0.287776,0.726485,0.287727,0.729252
...,...,...,...,...,...,...
4995,0.014301,1346.0,0.537232,0.991172,0.548562,0.999898
4996,0.004599,3222.0,0.610568,0.266016,0.606904,0.263237
4997,0.006412,9526.0,0.798692,0.359378,0.804831,0.361231
4998,0.002582,521.0,0.576455,0.549457,0.578345,0.547698


In [12]:
#remove the nearest point
deleted_result=numpy.delete(combined_x_y_arrays, int(result["indexes"][0]),0)

#store the nearest point
storage=numpy.array(result["dist"])
storage

array([0.02404103, 0.00651547, 0.00266997, ..., 0.00423273, 0.00492169,
       0.00360531])

In [13]:
for k in range(10):
    #
    kd_result = do_kdtree(deleted_result,points_list)
    df=pd.DataFrame(kd_result, index=rows)

    #
    nearest_points=combined_x_y_arrays[kd_result[1]]
    nearest_points=nearest_points.transpose()

    #
    df=df.append(pd.DataFrame(nearest_points , index=["nearest_point x","nearest_point y"]))
    point_xy=pd.DataFrame(data=points, index=["x","y"])
    result = df.append(point_xy).transpose()
    
    #remove the nearest point
    deleted_result=numpy.delete(deleted_result, int(result["indexes"][0]),0)
    #store the nearest point
    storage=numpy.append(storage, result["dist"][0])
    